# On The Free Energy Penalty of Cavity Formation In Salt Solutions: Rethinking the Terms “Kosmotropic” and “Chaotropic”.

Stefan Hervø-Hansen<sup>a,*</sup> and Nobuyuki Matubayasi<sup>a,*</sup>.<br><br>
<sup>a</sup> Division of Chemical Engineering, Graduate School of Engineering Science, Osaka University, Toyonaka, Osaka 560-8531, Japan.<br>
<sup>*</sup> To whom correspondence may be addressed: stefan@cheng.es.osaka-u.ac.jp and nobuyuki@cheng.es.osaka-u.ac.jp.

## Simulations

### Introduction

### Methods & Materials

### References



## Import of Python Modules & Auxiliary Functions

In [3]:
# Notebook dependent libs
import numpy as np
import matplotlib.pyplot as plt
import os, time
from shutil import copyfile
from distutils.spawn import find_executable
import parmed as pmd
import mdtraj as md

# Check for external programs
if None in [find_executable('packmol')]:
    print('WARNING: External program missing!')

# homedir = !pwd
# homedir = homedir[0]

homedir = '/home/lin/data/Cavity-Formation'
mddir = '/home/lin/data/Cavity-Formation/md'
print(homedir)

/data2/kasa/packages/amber/amber18/lib/python2.7/site-packages/ParmEd-3.2.0-py2.7-linux-x86_64.egg/parmed/utils/netcdf.py:770: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if typecode is not 'c':


/home/lin/data/Cavity-Formation


## Molecular Dynamics Simulations
### Simulation settings

In [9]:
salts = {
    'NaF': {'cation': 'Na', 'anion': 'F', 'rcation': 'NA', 'ranion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl', 'rcation': 'NA', 'ranion': 'CL'},
    'NaI': {'cation': 'Na', 'anion': 'I', 'rcation': 'NA', 'ranion': 'I'}
}

solutes = {
    'ethane': {'resname': 'ETH'},
    # 'n-butane': {'resname': 'BUT'},
    # 'propane': {'resname': 'PRP'},
    'n-hexane': {'resname': 'HEX'},
}

concentrations = {
#     0.0:{'N_solute':1,'N_water':10000,'N_cation':0,'N_anion':0},
    # 0.25:{'N_solute':1,'N_water':10000,'N_cation':45,'N_anion':45},
    0.5:{'N_solute':1,'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_solute':1,'N_water':10000,'N_cation':135,'N_anion':135},
    # 1.00:{'N_solute':1,'N_water':10000,'N_cation':180,'N_anion':180}
}

insolute = True
# insolute = False

### For salt in water

#### Generation of initial configuration using Packmol

In [13]:
packmol_script="""
tolerance 2.0
filetype pdb
output system.pdb

add_box_sides 1.0

structure ../../../../../PDB_files/Ions/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../PDB_files/Ions/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../PDB_files/OPC.pdb
        number {N_water}
        inside cube 0. 0. 0. 66.8 .
end structure
"""

topology_script = """
; topol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
Na            11  22.989769  0.00000000  A        0.25658      0.0971555
Ag            47 107.868220  0.00000000  A        0.26531      0.1492163
Br            35  79.904100  0.00000000  A        0.41195      2.7048107
Cl            17  35.453200  0.00000000  A        0.38309      2.1820915
Cs            55 132.905452  0.00000000  A        0.35458      1.5913927
Cu            29  63.546300  0.00000000  A        0.21399      0.0046986
F              9  18.998403  0.00000000  A        0.30647      0.6360531
I             53 126.904473  0.00000000  A        0.45846      3.3280127
K             19  39.098310  0.00000000  A        0.30968      0.6903724
Li             3   6.941200  0.00000000  A        0.23252      0.0218984
Rb            37  85.467830  0.00000000  A        0.32108      0.8985523
Tl            81 204.383320  0.00000000  A        0.32874       1.049263
OW            OW      0.0000  0.0000  A   3.16655e-01  8.903586e-01
HW            HW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
MW            MW      0.0000  0.0000  A   0.00000e+00  0.00000e+00

#include "../../../../../Force_fields/OPC.itp"
#include "../../../../../Force_fields/ions.itp"

[ system ]
{cation}{anion} in water

[ molecules ]
; Compound nmols
{cation} {N_cation}
{anion} {N_anion}
HOH 10000
"""

In [14]:

%cd /home/lin/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md/salt_water'
outputdir = '/home/lin/Cavity-Formation/Simulations'

for salt,S in salts.items():
    for concentration,N in concentrations.items():
        # making directory
        %mkdir {mddir}/{salt}
        %mkdir {outputdir}/{salt}
        %mkdir {mddir}/{salt}/{concentration} 
        %mkdir {outputdir}/{salt}/{concentration}
        folder_names = ['sys','em','nvt','npt','md']
        for folder_name in folder_names:
            %mkdir {mddir}/{salt}/{concentration}/{folder_name}
        
        %cd {mddir}/{salt}/{concentration}/sys
        # Generate initial configuration using packmol
        with open('packmol.inp', 'w') as f:
            f.write(packmol_script.format(homedir=homedir,cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()
        !packmol < packmol.inp

        # Convert packmol pdb to gromacs format
        init = pmd.load_file('system.pdb')
        init.save('system.gro')

        # Write gromacs topology
        with open('topol.top', 'w') as f:
            f.write(topology_script.format(cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                          N_anion=N['N_anion'],N_cation=N['N_cation']))
        f.close()

/data4/lin/Cavity-Formation/Simulations
mkdir: cannot create directory ‘/home/lin/Cavity-Formation/Simulations/NaF’: File exists
mkdir: cannot create directory ‘/home/lin/Cavity-Formation/Simulations/NaF/0.5’: File exists
/data4/lin/Cavity-Formation/md/salt_water/NaF/0.5/sys

################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 18.169 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Will print BOX SIDE informations. 
  Will sum    1.0000000000000000       to each side length on print
  Seed for random number generator

#### Submit script

In [3]:
header = """#!/bin/bash
#PBS -l nodes=1:nu-g:ppn=36

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2021.4-intel-gpu.rc
"""

add_script = """
###################################################################

cd {mddir}/salt_water/{salt}/{concentration}

#em

cd em

gmx_mpi grompp -f ../../../../../MDP_files/em.mdp -c ../sys/system.gro -p ../sys/topol.top -o solution_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_min -ntomp 1

cd ..

#NVT

cd nvt

gmx_mpi grompp -f ../../../../../MDP_files/nvt.mdp -c ../em/solution_min.gro -p ../sys/topol.top -o solution_nvt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_nvt -ntomp 1

cd ..

#NPT

cd npt

gmx_mpi grompp -f ../../../../../MDP_files/npt.mdp -c ../nvt/solution_nvt.gro -p ../sys/topol.top -o solution_npt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_npt -x -ntomp 1

cd ../

#md

cd md

gmx_mpi grompp -f ../../../../../MDP_files/md_10.mdp -c ../npt/solution_npt.gro -p ../sys/topol.top -o solution_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_run -x -s solution_run.tpr -ntomp 1

cd ../..

DATE=`date`
echo -e '\n'$DATE'\nendcalc {mddir}/salt_water/{salt}/{concentration}/md' >> /home/lin/Cavity-Formation/Reserch_notes.txt

###################################################################
"""

In [ ]:
homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md'
outputdir = '/home/lin/Cavity-Formation/Simulations'

tag = 0
for concentration,N in concentrations.items():
    for salt,S in salts.items():
        add = add_script.format(mddir=mddir,salt=salt,concentration=concentration)
        if tag == 0:
            script = header + add
        else:
            script = script + add
        tag = tag+1

#mdを行う
%cd {mddir}/salt_water
with open('erun.sh', 'w') as f:
    f.write(script)
f.close()
print(salt)
# !qsub run.sh
        

### For hydrocarbon and salt in water

In [5]:
packmol_script="""
tolerance 2.0
filetype pdb
output system.pdb

add_box_sides 1.0

structure ../../../../../../PDB_files/n-alkanes/{solute}.pdb
        number {N_solute}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../../PDB_files/Ions/{cation}.pdb
        number {N_cation}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../../PDB_files/Ions/{anion}.pdb
        number {N_anion}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../../PDB_files/OPC.pdb
        number {N_water}
        inside cube 0. 0. 0. 66.8 .
end structure
"""

topology_script = """
; topol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
; ions
Na            11  22.989769  0.00000000  A        0.25658      0.0971555
Ag            47 107.868220  0.00000000  A        0.26531      0.1492163
Br            35  79.904100  0.00000000  A        0.41195      2.7048107
Cl            17  35.453200  0.00000000  A        0.38309      2.1820915
Cs            55 132.905452  0.00000000  A        0.35458      1.5913927
Cu            29  63.546300  0.00000000  A        0.21399      0.0046986
F              9  18.998403  0.00000000  A        0.30647      0.6360531
I             53 126.904473  0.00000000  A        0.45846      3.3280127
K             19  39.098310  0.00000000  A        0.30968      0.6903724
Li             3   6.941200  0.00000000  A        0.23252      0.0218984
Rb            37  85.467830  0.00000000  A        0.32108      0.8985523
Tl            81 204.383320  0.00000000  A        0.32874       1.049263
; water
OW            OW      0.0000  0.0000  A   3.16655e-01  8.903586e-01
HW            HW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
MW            MW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
; alkanes
C1             6  12.010780  0.00000000  A     0.33795318     0.45538912
H1             1   1.007947  0.00000000  A     0.26445434    0.066021356

#include "../../../../../../Force_fields/{solute}.itp"
#include "../../../../../../Force_fields/ions.itp"
#include "../../../../../../Force_fields/OPC.itp"

[ system ]
{solute} in {cation}{anion} solution

[ molecules ]
; Compound nmols
{resname} {N_solute}
{rcation} {N_cation}
{ranion} {N_anion}
HOH 10000
"""

In [28]:
salts = {
    'NaF': {'cation': 'Na', 'anion': 'F', 'rcation': 'NA', 'ranion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl', 'rcation': 'NA', 'ranion': 'CL'},
    'NaI': {'cation': 'Na', 'anion': 'I', 'rcation': 'NA', 'ranion': 'I'}
}

solutes = {
    # 'ethane': {'resname': 'ETH'},
    # 'n-butane': {'resname': 'BUT'},
    # 'propane': {'resname': 'PRP'},
    'n-hexane': {'resname': 'HEX'},
}

concentrations = {
#     0.0:{'N_solute':1,'N_water':10000,'N_cation':0,'N_anion':0},
    # 0.25:{'N_solute':1,'N_water':10000,'N_cation':45,'N_anion':45},
    0.5:{'N_solute':1,'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_solute':1,'N_water':10000,'N_cation':135,'N_anion':135},
    # 1.00:{'N_solute':1,'N_water':10000,'N_cation':180,'N_anion':180}
}

insolute = True
# insolute = False

In [15]:
%cd /data4/lin/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/data4/lin/Cavity-Formation/md'
outputdir = '/data4/lin/Cavity-Formation/Simulations'

for solute,L in solutes.items():
    for salt,S in salts.items():
        for concentration,N in concentrations.items():
            # make directory
            new_dir_path = f'{mddir}/solute_salt_water/{solute}/{salt}/{concentration}'
            if not os.path.exists(new_dir_path):
                os.makedirs(new_dir_path)
            folder_names = ['sys','em','nvt','npt','md']
            for folder_name in folder_names:
                %mkdir {new_dir_path}/{folder_name}
                        
            %cd {new_dir_path}/sys
            # Generate initial configuration using packmol
            if not os.path.exists(f'{new_dir_path}/sys/packmol.inp'):
                with open('packmol.inp', 'w') as f:
                    f.write(packmol_script.format(solute=solute,resname=L['resname'],homedir=homedir,cation=S['cation'], anion=S['anion'],N_water=N['N_water'],
                                                N_anion=N['N_anion'],N_cation=N['N_cation'],N_solute=N['N_solute']))
                f.close()
                !packmol < packmol.inp

                # Convert packmol pdb to gromacs format
                init = pmd.load_file('system.pdb')
                init.save('system.gro')

            # Write gromacs topology
            with open('topol.top', 'w') as f:
                f.write(topology_script.format(solute=solute,resname=L['resname'],cation=S['cation'],anion=S['anion'],rcation=S['rcation'], ranion=S['ranion'],N_water=N['N_water'],
                                            N_anion=N['N_anion'],N_cation=N['N_cation'],N_solute=N['N_solute']))
            f.close()

/data4/lin/Cavity-Formation/Simulations
mkdir: cannot create directory ‘/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/sys’: File exists
mkdir: cannot create directory ‘/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/em’: File exists
mkdir: cannot create directory ‘/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/nvt’: File exists
mkdir: cannot create directory ‘/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/npt’: File exists
mkdir: cannot create directory ‘/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/md’: File exists
/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaF/0.5/sys
/data4/lin/Cavity-Formation/md/solute_salt_water/ethane/NaCl/0.5/sys

################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part o

#### Submit script

In [16]:
header = """#!/bin/bash
#PBS -l nodes=1:nu-g:ppn=36

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2021.4-intel-gpu.rc

"""

add_script = """
###################################################################

cd {mddir}/solute_salt_water/{solute}/{salt}/{concentration}

#em

cd em

gmx_mpi grompp -f ../../../../../../MDP_files/em.mdp -c ../sys/system.gro -p ../sys/topol.top -o solution_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_min -ntomp 1

cd ..

#NVT

cd nvt

gmx_mpi grompp -f ../../../../../../MDP_files/nvt.mdp -c ../em/solution_min.gro -p ../sys/topol.top -o solution_nvt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_nvt -ntomp 1

cd ..

#NPT

cd npt

gmx_mpi grompp -f ../../../../../../MDP_files/npt.mdp -c ../nvt/solution_nvt.gro -p ../sys/topol.top -o solution_npt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_npt -x -ntomp 1

cd ../

#md

cd md

gmx_mpi grompp -f ../../../../../../MDP_files/md.mdp -c ../npt/solution_npt.gro -p ../sys/topol.top -o solution_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_run -x -s solution_run.tpr -ntomp 1

cd ../..

DATE=`date`
echo -e '\n'$DATE'\nendcalc {mddir}/solute_salt_water/{solute}/{salt}/{concentration}' >> /home/lin/Cavity-Formation/Reserch_notes.txt

###################################################################
"""

In [29]:
homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md'
outputdir = '/home/lin/Cavity-Formation/Simulations'

tag = 0
for solute,L in solutes.items():
    for concentration,N in concentrations.items():
        for salt,S in salts.items():
            add = add_script.format(mddir=mddir,solute=solute,salt=salt,concentration=concentration)
            if tag == 0:
                script = header + add
            else:
                script = script + add
            tag = tag+1
            print (f'{solute} {salt} {concentration}')

test = False

if not test:
    #mdを行う
    os.chdir(f'{mddir}/solute_salt_water')
    with open('run1.sh', 'w') as f:
        f.write(script)
    f.close()
    os.system('qsub run1.sh')
else:
    print (script)

        

n-hexane NaF 0.5
n-hexane NaCl 0.5
n-hexane NaI 0.5
/data4/lin/Cavity-Formation/md/solute_salt_water


### For hydrocarbon in water

In [33]:
packmol_script="""
tolerance 2.0
filetype pdb
output system.pdb

add_box_sides 1.0

structure ../../../../../PDB_files/n-alkanes/{solute}.pdb
        number {N_solute}
        inside cube 0. 0. 0. 66.8 .
end structure

structure ../../../../../PDB_files/OPC.pdb
        number {N_water}
        inside cube 0. 0. 0. 66.8 .
end structure
"""

topology_script = """
; topol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
; water
OW            OW      0.0000  0.0000  A   3.16655e-01  8.903586e-01
HW            HW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
MW            MW      0.0000  0.0000  A   0.00000e+00  0.00000e+00
; alkanes
C1             6  12.010780  0.00000000  A     0.33795318     0.45538912
H1             1   1.007947  0.00000000  A     0.26445434    0.066021356

#include "../../../../../Force_fields/{solute}.itp"
#include "../../../../../Force_fields/OPC.itp"

[ system ]
{solute} in water

[ molecules ]
; Compound nmols
{resname} {N_solute}
HOH 10000
"""

In [31]:
salts = {
    'NaF': {'cation': 'Na', 'anion': 'F', 'rcation': 'NA', 'ranion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl', 'rcation': 'NA', 'ranion': 'CL'},
    'NaI': {'cation': 'Na', 'anion': 'I', 'rcation': 'NA', 'ranion': 'I'}
}

solutes = {
    'ethane': {'resname': 'ETH'},
    # 'n-butane': {'resname': 'BUT'},
    # 'propane': {'resname': 'PRP'},
    'n-hexane': {'resname': 'HEX'},
}

concentrations = {
#     0.0:{'N_solute':1,'N_water':10000,'N_cation':0,'N_anion':0},
    # 0.25:{'N_solute':1,'N_water':10000,'N_cation':45,'N_anion':45},
    0.5:{'N_solute':1,'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_solute':1,'N_water':10000,'N_cation':135,'N_anion':135},
    # 1.00:{'N_solute':1,'N_water':10000,'N_cation':180,'N_anion':180}
}

insolute = True
# insolute = False

In [36]:
%cd /data4/lin/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/data4/lin/Cavity-Formation/md'
outputdir = '/data4/lin/Cavity-Formation/Simulations'

for solute,L in solutes.items():
    for salt,S in salts.items():
        for concentration,N in concentrations.items():
            # make directory
            new_dir_path = f'{mddir}/solute_water/{solute}/solution'
            folder_names = ['sys','em','nvt','npt','md']
            if not os.path.exists(f'{new_dir_path}/sys'):            
                for folder_name in folder_names:
                    os.makedirs(f'{new_dir_path}/{folder_name}')
                        
            %cd {new_dir_path}/sys
            # Generate initial configuration using packmol
            if not os.path.exists(f'{new_dir_path}/sys/system.gro'):
                with open('packmol.inp', 'w') as f:
                    f.write(packmol_script.format(solute=solute,resname=L['resname'],homedir=homedir,N_water=N['N_water'],N_solute=N['N_solute']))
                f.close()
                !packmol < packmol.inp

                # Convert packmol pdb to gromacs format
                init = pmd.load_file('system.pdb')
                init.save('system.gro')

            # Write gromacs topology
            with open('topol.top', 'w') as f:
                f.write(topology_script.format(solute=solute,resname=L['resname'],N_water=N['N_water'],N_solute=N['N_solute']))
            f.close()

/data4/lin/Cavity-Formation/Simulations
/data4/lin/Cavity-Formation/md/solute_water/ethane/solution/sys

################################################################################

 PACKMOL - Packing optimization for the automated generation of
 starting configurations for molecular dynamics simulations.
 
                           Included as part of Packmol Memgen
                                                              Version 18.169 

################################################################################

  Packmol must be run with: packmol < inputfile.inp 

  Userguide at: www.ime.unicamp.br/~martinez/packmol 

  Reading input file... (Control-C aborts)
  Will print BOX SIDE informations. 
  Will sum    1.0000000000000000       to each side length on print
  Seed for random number generator:      1234567
  Output file: system.pdb
  Reading coordinate file: ../../../../../PDB_files/n-alkanes/ethane.pdb
  Reading coordinate file: ../../../../../PDB_files/OPC.pd

#### Submit script

In [43]:
header = """#!/bin/bash
#PBS -l nodes=1:nu-03:ppn=36

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2021.4-intel-single.rc

"""

add_script = """
###################################################################

cd {mddir}/solute_water/{solute}/solution

#em

cd em

gmx_mpi grompp -f ../../../../../MDP_files/em.mdp -c ../sys/system.gro -p ../sys/topol.top -o solution_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_min -ntomp 1

cd ..

#NVT

cd nvt

gmx_mpi grompp -f ../../../../../MDP_files/nvt.mdp -c ../em/solution_min.gro -p ../sys/topol.top -o solution_nvt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_nvt -ntomp 1

cd ..

#NPT

cd npt

gmx_mpi grompp -f ../../../../../MDP_files/npt.mdp -c ../nvt/solution_nvt.gro -p ../sys/topol.top -o solution_npt.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_npt -x -ntomp 1

cd ../

#md

cd md

gmx_mpi grompp -f ../../../../../MDP_files/md.mdp -c ../npt/solution_npt.gro -p ../sys/topol.top -o solution_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solution_run -x -s solution_run.tpr -ntomp 1

cd ../..

DATE=`date`
echo -e '\n'$DATE'\nendcalc cd {mddir}/solute_water/{solute}/solution' >> /home/lin/Cavity-Formation/Reserch_notes.txt

###################################################################
"""

In [44]:
test = False

homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md'
outputdir = '/home/lin/Cavity-Formation/Simulations'

tag = 0
for solute,L in solutes.items():
    add = add_script.format(mddir=mddir,solute=solute)
    if tag == 0:
        script = header + add
    else:
        script = script + add
    tag = tag+1
    print (f'{solute}')

if not test:
    #mdを行う
    os.chdir(f'{mddir}/solute_water')
    with open('run.sh', 'w') as f:
        f.write(script)
    f.close()
    os.system('qsub run.sh')
else:
    print (script)

        

ethane
n-hexane


### For isolated hydrocarbon

In [47]:
salts = {
    'NaF': {'cation': 'Na', 'anion': 'F', 'rcation': 'NA', 'ranion': 'F'},
    'NaCl': {'cation': 'Na', 'anion': 'Cl', 'rcation': 'NA', 'ranion': 'CL'},
    'NaI': {'cation': 'Na', 'anion': 'I', 'rcation': 'NA', 'ranion': 'I'}
}

solutes = {
    'ethane': {'resname': 'ETH'},
    # 'n-butane': {'resname': 'BUT'},
    # 'propane': {'resname': 'PRP'},
    'n-hexane': {'resname': 'HEX'},
}

concentrations = {
#     0.0:{'N_solute':1,'N_water':10000,'N_cation':0,'N_anion':0},
    # 0.25:{'N_solute':1,'N_water':10000,'N_cation':45,'N_anion':45},
    0.5:{'N_solute':1,'N_water':10000,'N_cation':90,'N_anion':90},
#     0.75:{'N_solute':1,'N_water':10000,'N_cation':135,'N_anion':135},
    # 1.00:{'N_solute':1,'N_water':10000,'N_cation':180,'N_anion':180}
}

insolute = True
# insolute = False

In [49]:
topology_script = """
; solutetopol.top
[ defaults ]
; nbfunc comb-rule gen-pairs fudgeLJ fudgeQQ
1 2 yes 0.5 0.8333

[ atomtypes ]
; name    at.num    mass    charge ptype  sigma      epsilon
C1             6  12.010780  0.00000000  A     0.33795318     0.45538912
H1             1   1.007947  0.00000000  A     0.26445434    0.066021356

#include "../../../../../Force_fields/{solute}.itp"

[ system ]
isolated {solute}

[ molecules ]
; Compound nmols
{resname} 1
"""

In [50]:
%cd /data4/lin/Cavity-Formation/Simulations

homedir = '/home/lin'
mddir = '/data4/lin/Cavity-Formation/md'
outputdir = '/data4/lin/Cavity-Formation/Simulations'

for solute,L in solutes.items():
    for salt,S in salts.items():
        for concentration,N in concentrations.items():
            # make directory
            new_dir_path = f'{mddir}/solute_water/{solute}/solute'
            folder_names = ['sys','em','md']
            if not os.path.exists(f'{new_dir_path}/sys'):            
                for folder_name in folder_names:
                    os.makedirs(f'{new_dir_path}/{folder_name}')
                        
            os.chdir(f'{new_dir_path}/sys')

            # Write gromacs topology
            with open('solutetopol.top', 'w') as f:
                f.write(topology_script.format(solute=solute,resname=L['resname']))
            f.close()

/data4/lin/Cavity-Formation/Simulations


#### Submit script

In [56]:
header = """#!/bin/bash
#PBS -l nodes=1:oz01:ppn=24

if [ "$PBS_O_WORKDIR" != "" ];then
  cd $PBS_O_WORKDIR
fi
NUMNODES=`wc -l < $PBS_NODEFILE`

source /data2/APPLI/packages/gromacs/gromacs-2021.4-intel-single.rc

"""

add_script = """
################isolated solute################

cd /home/lin/Cavity-Formation/md/solute_water/{solute}/solute

cd solute

cd em

gmx_mpi grompp -f ../../../../../MDP_files/solute_min.mdp -c ../../../../../PDB_files/test/b-{solute}.gro -p ../sys/solutetopol.top -o solute_min.tpr -maxwarn 1
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solute_min

cd ..

cd md

gmx_mpi grompp -f ../../../../../MDP_files/solute_run.mdp -c ../em/solute_min.gro -e ../em/solute_min.edr -p ../sys/solutetopol.top -o solute_run.tpr
mpirun -machinefile $PBS_NODEFILE -np $NUMNODES gmx_mpi mdrun -v -deffnm solute_run -x solute_run.xtc

cd ..

DATE=`date`
echo -e '\n'$DATE'\nendcalc /home/lin/Cavity-Formation/md/solute_water/{solute}/solute' >> /home/lin/Cavity-Formation/Reserch_notes.txt

################isolated solute################
"""

In [57]:
test = False

homedir = '/home/lin'
mddir = '/home/lin/Cavity-Formation/md'
outputdir = '/home/lin/Cavity-Formation/Simulations'

tag = 0
for solute,L in solutes.items():
    add = add_script.format(mddir=mddir,solute=solute)
    if tag == 0:
        script = header + add
    else:
        script = script + add
    tag = tag+1
    print (f'{solute}')

if not test:
    #mdを行う
    os.chdir(f'{mddir}/solute_water')
    with open('run1.sh', 'w') as f:
        f.write(script)
    f.close()
    os.system('qsub run1.sh')
else:
    print (script)

        

ethane
n-hexane


# RESEARCH NOTES
_will be deleted when the study is finished_